# Simularium Conversion Tutorial : CytoSim Data

In [1]:
from IPython.display import Image
import numpy as np
from simulariumio import Converter

This notebook provides example python code for converting your own simulation trajectories into the format consumed by the Simularium Viewer. It creates a .simularium JSON file which you can drag and drop onto the viewer like this:

![title](img/drag_drop.gif)

***
## Prepare your spatial data

The Simularium custom data Converter consumes spatiotemporal data from CytoSim. 

To see how to generate Cytosim output .txt files, check Cytosim documentation here: https://gitlab.com/f.nedelec/cytosim-stable/-/blob/master/doc/sim/report.md
* for Fibers, use the command `./report fiber:points > fiber_points.txt`, which will create `fiber_points.txt`
* for Solids, use the command `./report solid > solids.txt`, which will create `solids.txt`
* for Singles, use the command `./report single:position > singles.txt`, which will create `singles.txt`
* for Couples, use the command `./report couple:state > couples.txt`, which will create `couples.txt`
    * in some versions of Cytosim, state is not a reporting option. In this case you can use `./report couple:[name of your couple] > couples_[name of your couple].txt` and provide a filepath for each type of couple in your data. If this is necessary, you should also check the position XYZ columns in your `couples.txt` file and override **position_indices** if they aren't at [2, 3, 4]

The Converter requires the following data (except for fields marked as optional):
* **box_size** : *np.ndarray (shape = [3])*
    * A numpy ndarray containing the XYZ dimensions of the simulation bounding volume
        * *for now we only support rectangular bounds, but will be adding support for spheres soon*
* **data** : *Dict[str, Any]*
    * **fibers** : *Dict[str, Any]* (optional)
        * **filepath** : *str*
            * path to fiber_points.txt
        * **agents** : *Dict[str, Any]* (optional)
            * **[agent type index from Cytosim data]** : *Dict[str, Any]*
                * the type index from Cytosim data mapped to display names for each type of fiber
                * **name** : *str* (optional)
                    * the display name for this type of fiber
                    * Default: "fiber[agent type index from Cytosim data]"
    * **solids** : *Dict[str, Any]* (optional)
        * **filepath** : *str*
            * path to solids.txt
        * **agents** : *Dict[str, Any]* (optional)
            * **[agent type index from Cytosim data]** : *Dict[str, Any]*
                * the type index from Cytosim data mapped to display names and radii for each type of solid
                * **name** : *str* (optional)
                    * the display name for this type of solid
                    * Default: "solid[agent type index from Cytosim data]"
                * **radius** : *float* (optional)
                    * the radius for this type of solid
                    * Default: 1.0
        * **position_indices** : *List[int]* (optional)
            * the columns in Cytosim's reports are not always consistent, use this to override them if your output file has different column indices for position XYZ
            * Default: [2, 3, 4]
    * **singles** : *Dict[str, Any]* (optional)
        * **filepath** : *str*
            * path to singles.txt
        * **agents** : *Dict[str, Any]* (optional)
            * **[agent type index from Cytosim data]** : *Dict[str, Any]*
                * the type index from Cytosim data mapped to display names and radii for each type of single
                * **name** : *str* (optional)
                    * the display name for this type of single
                    * Default: "single[agent type index from Cytosim data]"
                * **radius** : *float* (optional)
                    * the radius for this type of single
                    * Default: 1.0
        * **position_indices** : *List[int]* (optional)
            * the columns in Cytosim's reports are not always consistent, use this to override them if your output file has different column indices for position XYZ
            * Default: [2, 3, 4]
    * **couples** : *Dict[str, Any]* (optional)
        * **filepath** : *str*
            * path to couples.txt
        * **agents** : *Dict[str, Any]* (optional)
            * **[agent type index from Cytosim data]** : *Dict[str, Any]*
                * the type index from Cytosim data mapped to display names and radii for each type of couple
                * **name** : *str* (optional)
                    * the display name for this type of couple
                    * Default: "couple[agent type index from Cytosim data]"
                * **radius** : *float* (optional)
                    * the radius for this type of couple
                    * Default: 1.0
        * **position_indices** : *List[int]* (optional)
            * the columns in Cytosim's reports are not always consistent, use this to override them if your output file has different column indices for position XYZ
            * Default: [2, 3, 4]
    
Optionally, you can also include:
* **scale_factor** : *float*
    * A multiplier for the Cytosim scene, use if visualization is too large or small
    * Default: 1.0
* **plots** : *Dict[str, Any]*
    * An object containing plot data already in Simularium format

In [2]:
# box_size = 2.

# example_data = {
#     "box_size" : np.array([box_size, box_size, box_size]),
#     "data" : {
#         "fibers" : {
#             "filepath" : "../simulariumio/tests/data/cytosim/fiber_test/fiber_points.txt",
#             "agents" : {
#                 "0" : {
#                     "name" : "microtubule"
#                 }
#             }
#         },
#     },
#     "scale_factor" : 100.0
# }

box_size = 2.

example_data = {
    "box_size" : np.array([box_size, box_size, box_size]),
    "data" : {
        "fibers" : {
            "filepath" : "../simulariumio/tests/data/cytosim/aster_pull3D_couples_actin_solid/fiber_points.txt",
            "agents" : {
                "1" : {
                    "name" : "microtubule"
                },
                "2" : {
                    "name" : "actin"
                }
            }
        },
        "solids" : {
            "filepath" : "../simulariumio/tests/data/cytosim/aster_pull3D_couples_actin_solid/solids.txt",
            "agents" : {
                "1" : {
                    "name" : "aster",
                    "radius" : 0.1
                },
                "2" : {
                    "name" : "vesicle",
                    "radius" : 0.1
                }
            }
        },
        "singles" : {
            "filepath" : "../simulariumio/tests/data/cytosim/aster_pull3D_couples_actin_solid/singles.txt",
            "agents" : {
                "1" : {
                    "name" : "dynein",
                    "radius" : 0.01
                },
                "2" : {
                    "name" : "kinesin",
                    "radius" : 0.01
                }
            }
        },
        "couples" : {
            "filepath" : "../simulariumio/tests/data/cytosim/aster_pull3D_couples_actin_solid/couples.txt",
            "agents" : {
                "1" : {
                    "name" : "motor complex",
                    "radius" : 0.02
                }
            },
            "position_indices" : [3, 4, 5]
        }
    },
    "scale_factor" : 100.0
}

## Convert and save as .simularium JSON file

Once your data is shaped like in the `example_data` object, you can use the converter to generate the file at the given path:

In [3]:
Converter(example_data, "cytosim").write_JSON("/Users/blairl/Desktop/cytosim_example")

## Visualize in the Simularium viewer

In your browser, either Firefox or Chrome, navigate to https://staging.agentviz.allencell.org/ and drag your file onto the center viewer window. 

*For now you'll first have to choose an example trajectory and close the load window. Once the example trajectory loads, you can drop your own file in to replace it. We'll fix this soon :)*